In [1]:
import os
print(os.getcwd())

c:\Users\UTENTE\Git Repos\Chicken-Disease-Classification--Project\research


In [3]:
os.chdir("../")
print(os.getcwd())

c:\Users\UTENTE\Git Repos\Chicken-Disease-Classification--Project


In [25]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [5]:
from src.cnnClassifier.utils.common import read_yaml, create_directories
from src.cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

In [26]:

class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_callback_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(model_ckpt_callback_dir),
            Path(config.tensorboard_root_log_dir),
        ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [14]:
import time
import os
import tensorflow as tf

In [27]:
class PrepareCallback:
    def __init__(self, config:PrepareCallbackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,
                     f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [ ]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callback_config)    
    callbacks_list = prepare_callbacks.get_tb_ckpt_callbacks()



except Exception as e:
    raise e

[2025-05-12 18:21:25,642: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-12 18:21:25,646: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-12 18:21:25,646: INFO: common: created directory at: artifacts]
[2025-05-12 18:21:25,652: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-05-12 18:21:25,653: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [29]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2025-05-12-18-23-03'